In [1]:
import numpy as np
import pandas as pd

## groupby

In [3]:
df = pd.DataFrame({'key1':list('aabba'),
                  'key2': ['one','two','one','two','one'],
                  'data1': np.random.randn(5),
                  'data2': np.random.randn(5)})
df

,data1,data2,key1,key2
0,-1.304072,0.602548,a,one
1,0.892170,-0.386406,a,two
2,0.457713,-1.045518,b,one
3,1.123721,-0.057502,b,two
4,-1.136373,1.305967,a,one


In [20]:
grouped = df.groupby(df['key1'])
grouped.mean()

,data1,data2
key1,,
a,-0.516091,0.50737
b,0.790717,-0.55151


In [21]:
grouped = df['data1'].groupby(df['key1'])
grouped.mean()

key1
a   -0.516091
b    0.790717
Name: data1, dtype: float64

In [23]:
# 按多个键分组
grouped = df.groupby([df['key1'],df['key2']])
grouped.mean()

data1     data2
key1 key2                    
a    one  -1.220222  0.954258
     two   0.892170 -0.386406
b    one   0.457713 -1.045518
     two   1.123721 -0.057502

In [25]:
# 以上的分组键均为Series，实际上分组键可以是任何长度适当的数组
states=np.array(['Ohio','California','California','Ohio','Ohio'])
years=np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

California  2005    0.892170
            2006    0.457713
Ohio        2005   -0.090175
            2006   -1.136373
Name: data1, dtype: float64

In [27]:
# 可以看出没有key2列，因为df[‘key2’]不是数值数据，所以被从结果中移除。
# 默认情况下，所有数值列都会被聚合，虽然有时可能被过滤为一个子集。
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.516091,0.50737
b,0.790717,-0.55151


## 对分组进行迭代

In [33]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -1.304072  0.602548    a  one
1  0.892170 -0.386406    a  two
4 -1.136373  1.305967    a  one
b
      data1     data2 key1 key2
2  0.457713 -1.045518    b  one
3  1.123721 -0.057502    b  two


In [34]:
# name就是groupby中的key1的值，group就是要输出的内容。 
# 同理：
for (k1,k2),group in df.groupby(['key1','key2']):
    print ('===k1,k2:')
    print (k1,k2)
    print ('===k3:')
    print (group)

===k1,k2:
a one
===k3:
      data1     data2 key1 key2
0 -1.304072  0.602548    a  one
4 -1.136373  1.305967    a  one
===k1,k2:
a two
===k3:
     data1     data2 key1 key2
1  0.89217 -0.386406    a  two
===k1,k2:
b one
===k3:
      data1     data2 key1 key2
2  0.457713 -1.045518    b  one
===k1,k2:
b two
===k3:
      data1     data2 key1 key2
3  1.123721 -0.057502    b  two


In [35]:
piece=dict(list(df.groupby('key1')))
piece

{'a':       data1     data2 key1 key2
 0 -1.304072  0.602548    a  one
 1  0.892170 -0.386406    a  two
 4 -1.136373  1.305967    a  one, 'b':       data1     data2 key1 key2
 2  0.457713 -1.045518    b  one
 3  1.123721 -0.057502    b  two}

In [38]:
# groupby默认是在axis=0上进行分组的，通过设置也可以在其他任何轴上进行分组.

grouped=df.groupby(df.dtypes, axis=1)
dict(list(grouped))

{dtype('float64'):       data1     data2
 0 -1.304072  0.602548
 1  0.892170 -0.386406
 2  0.457713 -1.045518
 3  1.123721 -0.057502
 4 -1.136373  1.305967, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

## 选取一个或者一组列

In [46]:
df

,data1,data2,key1,key2
0,-1.304072,0.602548,a,one
1,0.892170,-0.386406,a,two
2,0.457713,-1.045518,b,one
3,1.123721,-0.057502,b,two
4,-1.136373,1.305967,a,one


In [52]:
print(df.groupby(['key1','key2']).mean())

# 对于大数据，很多情况是只需要对部分列进行聚合
print(df.groupby(['key1','key2'])['data2'].mean())
df.groupby(['key1','key2'])['data2','data1'].mean()

              data1     data2
key1 key2                    
a    one  -1.220222  0.954258
     two   0.892170 -0.386406
b    one   0.457713 -1.045518
     two   1.123721 -0.057502
key1  key2
a     one     0.954258
      two    -0.386406
b     one    -1.045518
      two    -0.057502
Name: data2, dtype: float64


data2     data1
key1 key2                    
a    one   0.954258 -1.220222
     two  -0.386406  0.892170
b    one  -1.045518  0.457713
     two  -0.057502  1.123721

## 通过字典或者series进行分组

In [70]:
people=pd.DataFrame(np.random.randn(5,5),
                   columns=list('abcde'),
                   index=['Joe','Steve','Wes','Jim','Travis'])

people.ix[2:3,['b','c']]=np.nan #设置几个nan
people

,a,b,c,d,e
Joe,0.836659,-0.155809,-0.260933,-0.479236,-0.747557
Steve,-0.750791,-0.672957,-0.830508,0.835848,0.327191
Wes,0.935452,NaN,NaN,0.260250,1.839458
Jim,0.977290,-0.223177,1.648929,-0.895358,-1.537907
Travis,0.838842,0.643993,0.130966,0.822127,0.882899


In [72]:
mapping={'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}
by_column=people.groupby(mapping,axis=1)
by_column.sum()

,blue,red
Joe,-0.740169,-0.066708
Steve,0.005340,-1.096557
Wes,0.260250,2.774910
Jim,0.753572,-0.783794
Travis,0.953092,2.365734


In [78]:
# 如果不加axis=1, 则只会出现 a b c d e
mapping={'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}
by_column=people.groupby(mapping)
by_column.sum()

,a,b,c,d,e


In [80]:
# Series 也一样
map_series=pd.Series(mapping)
print(map_series)
people.groupby(map_series,axis=1).count()

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object


,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## 通过函数进行分组

In [86]:
# 相较于dic或者Series，python函数在定义分组关系映射时更有创意。
# 任何被当做分组键的函数都会在各个索引上被调用一次，其返回值就会被用作分组名称。
# 假设你按人名的长度进行分组，仅仅传入len即可
print(people)
people.groupby(len).sum()

               a         b         c         d         e
Joe     0.836659 -0.155809 -0.260933 -0.479236 -0.747557
Steve  -0.750791 -0.672957 -0.830508  0.835848  0.327191
Wes     0.935452       NaN       NaN  0.260250  1.839458
Jim     0.977290 -0.223177  1.648929 -0.895358 -1.537907
Travis  0.838842  0.643993  0.130966  0.822127  0.882899


,a,b,c,d,e
3,2.749401,-0.378987,1.387996,-1.114343,-0.446006
5,-0.750791,-0.672957,-0.830508,0.835848,0.327191
6,0.838842,0.643993,0.130966,0.822127,0.882899


In [87]:
# 将函数和数组、列表、字典、Series混合使用也不是问题，因为任何东西都会最终转换为数组
key_list=['one','one','one','two','two'] 
people.groupby([len,key_list]).sum()

a         b         c         d         e
3 one  1.772111 -0.155809 -0.260933 -0.218986  1.091901
  two  0.977290 -0.223177  1.648929 -0.895358 -1.537907
5 one -0.750791 -0.672957 -0.830508  0.835848  0.327191
6 two  0.838842  0.643993  0.130966  0.822127  0.882899

## 根据索引级别进行分组

In [88]:
# 层次化索引最方便的地方就在于他能够根据索引级别进行聚合。
# 要实现该目的，通过level关键字出入级别编号或者名称即可:
columns=pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]],names=['cty','tenor'])
hier_df=pd.DataFrame(np.random.randn(4,5),columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.841506 -0.318683 -0.727548 -0.102212  1.135224
1      0.656744 -0.371614 -0.186148 -0.991809  0.944900
2     -0.499534  0.083669 -0.464316 -0.131412 -1.253453
3     -0.496822  1.911844 -1.335649  0.436272  1.962672

In [89]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 数据聚合

### 调用自定义的聚合函数

In [110]:
print(df)
grouped = df['data1'].groupby(df['key1'])
for name,group in df['data1'].groupby(df['key1']):
    print ('===name:')
    print (name)
    print ('===group:')
    print (group)

# 自定义函数
def peak_to_peak(arr):
    return arr.max()-arr.min()
grouped.agg(peak_to_peak)

      data1     data2 key1 key2
0 -1.304072  0.602548    a  one
1  0.892170 -0.386406    a  two
2  0.457713 -1.045518    b  one
3  1.123721 -0.057502    b  two
4 -1.136373  1.305967    a  one
===name:
a
===group:
0   -1.304072
1    0.892170
4   -1.136373
Name: data1, dtype: float64
===name:
b
===group:
2    0.457713
3    1.123721
Name: data1, dtype: float64


key1
a    2.196242
b    0.666009
Name: data1, dtype: float64

### 面向列的多函数应用

In [ ]:
# 常用函数  count、sum、mean、median、std、var(方差)、
# min、max、prod（非NA值的积）、first（第一个非NA的值）、last

In [137]:
# 对Series或者DataFrame列的聚合运算实际是使用aggregate或者调用mean，std等方法。
# 下面对不同的列使用不同的聚合函数，或者一次应用多个函数
print(df)
grouped = df['data1'].groupby(df['key1'])

# 将函数名直接以字符串传入
print("1*"*10)
print(grouped.agg('mean') )

#如果传入一组函数，得到的df的列名就会以相应的函数命名
print("2*"*10)
print(grouped.agg(['mean','std',peak_to_peak]) )

# 如果传入的是(name, function)元组组成的列表，则各个元组的第一个元素将被用作df的列名
print("3*"*10)
print(grouped.agg([('0_+','mean'),('0.0',np.std),('=.=',np.std)]) )

# 对于df，可以定义一组用于全部列的函数,或在不同的列应用不同的函数 
print("4*"*10)
functions = ['mean','std',peak_to_peak]
print(grouped.agg(functions))

# 如果想对不同的列应用不同的函数, 具体的办法是想agg传入一个从列名映射到函数的字典
print("5*"*10)
grouped = df.groupby(df['key1'])
functions = {'data1':'mean', 'data2':'sum'}
grouped.agg(functions)

      data1     data2 key1 key2
0 -1.304072  0.602548    a  one
1  0.892170 -0.386406    a  two
2  0.457713 -1.045518    b  one
3  1.123721 -0.057502    b  two
4 -1.136373  1.305967    a  one
1*1*1*1*1*1*1*1*1*1*
key1
a   -0.516091
b    0.790717
Name: data1, dtype: float64
2*2*2*2*2*2*2*2*2*2*
          mean       std  peak_to_peak
key1                                  
a    -0.516091  1.222470      2.196242
b     0.790717  0.470939      0.666009
3*3*3*3*3*3*3*3*3*3*
           0_+       0.0       =.=
key1                              
a    -0.516091  1.222470  1.222470
b     0.790717  0.470939  0.470939
4*4*4*4*4*4*4*4*4*4*
          mean       std  peak_to_peak
key1                                  
a    -0.516091  1.222470      2.196242
b     0.790717  0.470939      0.666009
5*5*5*5*5*5*5*5*5*5*


,data1,data2
key1,,
a,-0.516091,1.522109
b,0.790717,-1.103019


## 分组级运算和转换

In [ ]:
# 聚合只是分组运算的一种，它是数据转换的特列。transform 和apply更牛叉

# transform会将一个函数应用到各个分组，然后将结果放在适当的位置. 如果各分组产生的标量值，则该标量值会被广播出去。
# transform也是有严格条件的特殊函数：传入的函数只能产生两种结果，
# 要么产生一个可以广播的标量值(如：np.mean), 要么产生一个相同大小的结果数组。

In [148]:
people=pd.DataFrame(np.random.randn(5,5),
                   columns=list('abcde'),
                   index=['Joe','Steve','Wes','Jim','Travis'])
people

,a,b,c,d,e
Joe,1.210137,0.547824,-0.128973,0.002002,-0.417625
Steve,-0.392114,0.416604,-0.805733,-0.012115,0.791804
Wes,1.294789,0.518447,-0.037716,-1.561559,-0.319690
Jim,1.467127,-0.267567,-0.536224,0.058921,0.336582
Travis,2.016760,-0.958386,-0.155097,0.869959,-0.595236


In [149]:
key=['one','two','one','two','one']
people.groupby(key).mean()

,a,b,c,d,e
one,1.507229,0.035962,-0.107262,-0.229866,-0.444184
two,0.537506,0.074519,-0.670979,0.023403,0.564193


In [150]:
people.groupby(key).transform(np.mean)
# 可以看到有很多与表2一样的值。

,a,b,c,d,e
Joe,1.507229,0.035962,-0.107262,-0.229866,-0.444184
Steve,0.537506,0.074519,-0.670979,0.023403,0.564193
Wes,1.507229,0.035962,-0.107262,-0.229866,-0.444184
Jim,0.537506,0.074519,-0.670979,0.023403,0.564193
Travis,1.507229,0.035962,-0.107262,-0.229866,-0.444184


In [151]:
def demean(arr):
    return arr-arr.mean()

demeaned=people.groupby(key).transform(demean)
demeaned

,a,b,c,d,e
Joe,-0.297092,0.511863,-0.021711,0.231868,0.026559
Steve,-0.929621,0.342086,-0.134754,-0.035518,0.227611
Wes,-0.212439,0.482485,0.069546,-1.331693,0.124493
Jim,0.929621,-0.342086,0.134754,0.035518,-0.227611
Travis,0.509531,-0.994348,-0.047835,1.099825,-0.151052


In [ ]:
# 原文：https://blog.csdn.net/youngbit007/article/details/54288603
# transform 与 apply 更多详自行百度

## 分位数和桶分析

In [ ]:
# cut and qcut与groupby结合起来，能轻松的对数据集的桶(bucket)或者分位数(quantile)分析。

In [162]:
frame=pd.DataFrame({'data1':np.random.randn(1000),
                   'data2': np.random.randn(1000)})
frame.head()

,data1,data2
0,-0.917789,-1.175998
1,-1.441893,-0.737888
2,1.252216,-0.569244
3,-0.235966,-0.904568
4,-1.906962,-0.121977


In [168]:
factor=pd.cut(frame.data1,4)
factor[:20]

0     (-1.786, -0.0166]
1     (-1.786, -0.0166]
2      (-0.0166, 1.753]
3     (-1.786, -0.0166]
4      (-3.562, -1.786]
5      (-0.0166, 1.753]
6      (-0.0166, 1.753]
7     (-1.786, -0.0166]
8      (-0.0166, 1.753]
9        (1.753, 3.522]
10     (-0.0166, 1.753]
11     (-0.0166, 1.753]
12     (-0.0166, 1.753]
13    (-1.786, -0.0166]
14    (-1.786, -0.0166]
15    (-1.786, -0.0166]
16     (-0.0166, 1.753]
17    (-1.786, -0.0166]
18    (-1.786, -0.0166]
19     (-0.0166, 1.753]
Name: data1, dtype: category
Categories (4, object): [(-3.562, -1.786] < (-1.786, -0.0166] < (-0.0166, 1.753] < (1.753, 3.522]]

In [172]:
def get_stats(group):
    return {'min':group.min(),'max':group.max(),'count':group.count(),'mean':group.mean()}
grouped=frame.data2.groupby(factor)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.562, -1.786]",28.0,2.458871,-0.207221,-2.845501
"(-1.786, -0.0166]",454.0,2.478393,-0.079693,-3.378321
"(-0.0166, 1.753]",484.0,3.058291,-0.030417,-2.822678
"(1.753, 3.522]",34.0,1.494383,-0.054172,-1.442812


In [174]:
# unstack函数将数据从"花括号结构（类似堆叠）"成"表格格式"
# 堆叠格式如下：
grouped.apply(get_stats)

data1                   
(-3.562, -1.786]   count     28.000000
                   max        2.458871
                   mean      -0.207221
                   min       -2.845501
(-1.786, -0.0166]  count    454.000000
                   max        2.478393
                   mean      -0.079693
                   min       -3.378321
(-0.0166, 1.753]   count    484.000000
                   max        3.058291
                   mean      -0.030417
                   min       -2.822678
(1.753, 3.522]     count     34.000000
                   max        1.494383
                   mean      -0.054172
                   min       -1.442812
Name: data2, dtype: float64

In [159]:
# 这些都是长度相等的桶，要根据样本分为数据量大小相等的桶，使用qcut即可.

# 长度相等的桶：区间大小相等 
# 大小相等的桶：数据点数量相等

In [176]:
grouping=pd.qcut(frame.data1,10,labels=False)#label=false即可值获取分位数的编号
grouped=frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.458871,-0.188458,-2.845501
1,100.0,2.279129,-0.094244,-2.038445
2,100.0,2.478393,-0.054182,-2.948644
3,100.0,2.390311,-0.082986,-2.111568
4,100.0,2.446017,-0.070393,-3.378321
5,100.0,2.214205,-0.016592,-2.492703
6,100.0,2.607421,-0.151927,-2.822678
7,100.0,2.227739,0.074419,-2.673179
8,100.0,2.633283,0.011705,-2.264143
